In [21]:
import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps

t_data = pd.read_csv("./dataset/train.csv")

In [22]:
t_data = t_data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis = 1)
t_data = t_data.dropna()
t_data.loc[t_data['Age'] <= 16, 'Age'] = 0
t_data.loc[(t_data['Age'] > 16) & (t_data['Age'] < 50), 'Age'] = 1
t_data.loc[t_data['Age'] >= 50, 'Age'] = 2
t_data.loc[t_data['Fare'] <=15.64585, 'Fare'] = 0
t_data.loc[(t_data['Fare'] > 15.64585) & (t_data['Fare'] < 33), 'Fare'] = 1
t_data.loc[t_data['Fare'] >= 33, 'Fare'] = 2
print(t_data.head())

   Survived  Pclass     Sex  Age  SibSp  Parch  Fare Embarked
0         0       3    male  1.0      1      0   0.0        S
1         1       1  female  1.0      1      0   2.0        C
2         1       3  female  1.0      0      0   0.0        S
3         1       1  female  1.0      1      0   2.0        S
4         0       3    male  1.0      0      0   0.0        S


In [23]:
t_data['Age'] = t_data['Age'].interpolate() #fixing missing values 

In [24]:
#Entropy of the entire dataset
entropy = 0
values = t_data['Survived'].unique()
for value in values:
    p = t_data['Survived'].value_counts()[value]/len(t_data.Survived)
    entropy = entropy + (-(p*log(p)))
print(entropy)

0.9735190023846809


In [25]:
def attribute_entropy(data, col):
    #print("Calculating att-ent")
    y = data['Survived'].unique()
    x = data[col].unique()
    entropy = 0
    for i in x:
        e = 0
        for j in y:
            num = len(data[col][data[col] == i][data['Survived'] == j])
            den = len(data[col][data[col] == i])
            p = num/(den+eps)
            e = e + (-(p*log(p+eps)))
        p2 = den/len(data)
        entropy = entropy + (-(p2*e))
    return abs(entropy)

In [26]:
print(attribute_entropy(t_data, 'Sex'))

0.7594106895489574


In [28]:
def find_max(d):
    #print("finding max")
    data = d.drop(['Survived'], axis = 1)
    att = []
    info_gain = []
    #print(data.keys())
    for key in data.keys()[:-1]:
        info_gain.append(entropy-attribute_entropy(d, key))
    print(info_gain)
    return data.keys()[np.argmax(info_gain)]
find_max(t_data)

[0.09400998456880671, 0.2141083128357235, 0.010380116836133935, 0.02486899311401769, 0.03068201178809904, 0.07107978429017991]


'Sex'

In [29]:
def get_subtable(df, node,value):
    #print('making a subtable')
    return df[df[node] == value].reset_index(drop=True)


def buildTree(data): 
    
    node = find_max(data)
    
    vals = np.unique(data[node])
    #print(vals)
    tree = {}
    tree[node] = {}
    for val in vals:
        subtable = get_subtable(data, node, val)
        #print((subtable.head()))
        
        if(len(set(subtable['Survived'])) == 1):
            tree[node][val] = subtable['Survived'].iloc[0]
        else:
            tree[node][val] = buildTree(subtable)
        print(tree)
    return tree

tree = buildTree(t_data)
#print(tree)


[0.09400998456880671, 0.2141083128357235, 0.010380116836133935, 0.02486899311401769, 0.03068201178809904, 0.07107978429017991]
[0.3935639322525507, 0.16686288142080752, 0.18007426653297298, 0.19844406116624813, 0.20877511570046214, 0.22489068097451237]
[0.7491920420395937, 0.7491920420395937, 0.7746533272115386, 0.7583461906935682, 0.7899514173750228, 0.7608355742576144]
[0.8404793537739826, 0.8404793537739826, 0.8866679665348144, 0.856384391096608, 0.8404793537739826, 0.8968206792351203]
[0.3818462238023541, 0.3818462238023541, 0.6878047166703956, 0.3818462238023541, 0.3818462238023541, 0.3818462238023541]
{'Age': {1.0: 1}}
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.0264809976153184

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

...

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413, -0.026480997615318413]
[-0.026480997615318413, -0.

RecursionError: maximum recursion depth exceeded while calling a Python object